In [512]:
import pandas as pd
import numpy as np

In [513]:
url = '/Users/aliciacaminero/Documents/CURSOS/DATA ANALYSIS BOOTCAMP IRONHACK/MINI-PROYECTOS/lego_harry_potter_sets.csv'
df = pd.read_csv(url)

In [514]:
# 1º vamos a eliminar las filas en las que todos sus datos son NaN
#df_hp.dropna(how='all', inplace=True) -> no hay

# Filtra las filas con NaN en las columnas especificadas
#df_nan = df[df[['PVP', 'Precio como Nuevo', 'Precio como Usado']].isna().any(axis=1)]

#df.dropna(subset=['Año de Lanzamiento', 'Precio como Nuevo', 'Precio como Usado'], inplace=True)


In [515]:
# Reemplaza valores no convertibles por NaN
df['Código del Set'] = pd.to_numeric(df['Código del Set'], errors='coerce')

# 2. Rellenar NaN con 0 (o eliminar filas, si prefieres) -> mejor a Int64
#df['Código del Set'] = df['Código del Set'].fillna(0)

df['Código del Set'] = df['Código del Set'].astype('Int64')

In [516]:
df['Precio como Nuevo'] = df['Precio como Nuevo'].str.lstrip('~')
df['Precio como Usado'] = df['Precio como Usado'].str.lstrip('~')

In [517]:
df.rename(columns={
    'PVP': 'PVP $',
    'Precio como Nuevo': 'Precio como Nuevo €',
    'Precio como Usado': 'Precio como Usado €'
}, inplace=True)

In [518]:
tasa_conversion = 1.10

# Limpiar la columna 'Precio como Nuevo' y transformar a dólares
df['Precio como Nuevo €'] = df['Precio como Nuevo €'].replace('Precio nuevo no disponible', np.nan)
df['Precio como Nuevo €'] = df['Precio como Nuevo €'].str.lstrip('€')
df['Precio como Nuevo $'] = df['Precio como Nuevo €'].astype(float) * tasa_conversion

# Limpiar la columna 'Precio como Usado' y transformar a dólares
df['Precio como Usado €'] = df['Precio como Usado €'].replace('Precio usado no disponible', np.nan)
df['Precio como Usado €'] = df['Precio como Usado €'].str.lstrip('€')
df['Precio como Usado $'] = df['Precio como Usado €'].astype(float) * tasa_conversion

# Adecuar columna PVP $
df['PVP $'] = df['PVP $'].replace('PVP no disponible', np.nan)
df['PVP $'] = df['PVP $'].str.lstrip('$')
df['PVP $'] = df['PVP $'].str.split(' | ').str[0]
df['PVP $'] = df['PVP $'].str.replace(',', '', regex=False)
df['PVP $'] = df['PVP $'].astype(float)


In [519]:
url2 = '/Users/aliciacaminero/Documents/CURSOS/DATA ANALYSIS BOOTCAMP IRONHACK/MINI-PROYECTOS/Tablas descargadas/lego_harry_potter_sets_dl.csv'
df_2 = pd.read_csv(url2)

In [520]:
df_reducido = df_2[['Number', 'YearFrom', 'SetName']]

In [521]:
df_reducido.loc[:, 'Number'] = pd.to_numeric(df_reducido['Number'],errors='coerce')

df_reducido.dropna(subset=['Number'], inplace=True)

df_reducido.loc[:, 'Number'] = df_reducido['Number'].astype(int)

/var/folders/8k/8w_q7kd116v1zrzns3x24cv80000gn/T/ipykernel_3896/69839567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reducido.dropna(subset=['Number'], inplace=True)


In [522]:
merged_HP_df = pd.merge(df, df_reducido, left_on='Código del Set', right_on='Number', how='left')

In [523]:
#merged_HP_df.loc[:, 'YearFrom'] = merged_HP_df['YearFrom'].fillna(0)  # hay que sustituir NaN por un cero si quieres que pase a int -> si quieres mantener NaN Int64
merged_HP_df['YearFrom'] = merged_HP_df['YearFrom'].astype('Int64')

In [524]:
merged_HP_df.drop(columns=['Precio como Nuevo €', 'Precio como Usado €', 'Number'], inplace=True)

In [525]:
# Crear la nueva columna 'FinalSetName' usando fillna -> para que principalmente aparezca SetName pero si aqui hay un NaN sea sustituido por Nombre del Set
merged_HP_df['FinalSetName'] = merged_HP_df['SetName'].fillna(merged_HP_df['Nombre del Set'])

In [526]:
merged_HP_df.drop(columns=['Nombre del Set', 'SetName'], inplace=True)

In [527]:
new_orden = ['Código del Set', 'FinalSetName', 'Año de Lanzamiento', 'YearFrom', 'PVP $', 'Precio como Nuevo $', 'Precio como Usado $', 'Valoración de Usuarios']

HP_df = merged_HP_df[new_orden]

In [528]:
HP_df.rename(columns={
    'FinalSetName': 'Nombre del Set',
    'Año de Lanzamiento': 'Fecha de Lanzamiento',
    'YearFrom': 'Año de Lanzamiento'
}, inplace=True)

In [529]:
tipos_columnas = merged_HP_df.dtypes

tipos_columnas

Código del Set             object
Año de Lanzamiento         object
PVP $                     float64
Valoración de Usuarios     object
Precio como Nuevo $       float64
Precio como Usado $       float64
YearFrom                    Int64
FinalSetName               object
dtype: object

In [530]:
HP_df

,Código del Set,Nombre del Set,Fecha de Lanzamiento,Año de Lanzamiento,PVP $,Precio como Nuevo $,Precio como Usado $,Valoración de Usuarios
0,4695,Mini Harry Potter Knight Bus,1 Apr 2004 - 31 Dec 2004 (9m),2004,NaN,23.914,8.866,✭✭✭✭✩ 3.6 64 ratings 11 reviews
1,4701,Sorting Hat,1 Sep 2001 - 31 Dec 2002 (1y 3m 29d),2001,7.0,87.659,13.563,✭✭✭✭✩ 3.4 154 ratings 19 reviews
2,4702,The Final Challenge,1 Sep 2001 - 31 Dec 2002 (1y 3m 29d),2001,10.0,44.979,17.688,✭✭✭✭✩ 3.6 154 ratings 14 reviews
3,4704,The Chamber of the Winged Keys,1 Sep 2001 - 31 Dec 2002 (1y 3m 29d),2001,20.0,91.707,35.123,✭✭✭✭✩ 3.7 95 ratings 9 reviews
4,4705,Snape's Class,1 Sep 2001 - 31 Dec 2002 (1y 3m 29d),2001,20.0,67.727,31.515,✭✭✭✭✩ 3.8 121 ratings 13 reviews
...,...,...,...,...,...,...,...,...
171,<NA>,Knight Bus,Año no disponible,<NA>,NaN,NaN,NaN,NaN
172,<NA>,Voldemort,Año no disponible,<NA>,0.0,NaN,NaN,NaN
173,<NA>,Golden Snitch,Año no disponible,<NA>,NaN,10.439,12.441,NaN
174,<NA>,Quidditch Tower (TRU Event Exclusive),Año no disponible,<NA>,NaN,NaN,24.178,1 review
